In [2]:
# output images from generator
# detect all humans in image with bbox
# compare that set of bboxes against the ones in the image with iou
# assign imageai bboxes to streetstyle bbox if there is exactly one >0.9 match (otherwise discard)
# save csv of id->new_bbox

In [3]:
from imageai.Detection import ObjectDetection
from pathlib import Path

import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

from tensorflow import keras

from PIL import Image, ImageDraw

from IPython.display import display

Using TensorFlow backend.


In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session( config=config )
keras.backend.set_session( sess )

In [5]:
ROOT = Path( "/home/lewis/Work/Employment/fellowshipai/" )
MODELS = ROOT/"platform-demos3/Fashion/models"
MODELS_RETINA = MODELS/"resnet50_coco_best_v2.0.1.h5"
DATA_ROOT = ROOT/"fashion/data/streetstyle"
PICKLES = DATA_ROOT/"pickles"
IMAGE_DIR = DATA_ROOT/"streetstyle27k"
CSV = DATA_ROOT/"ss27k_map.csv"
CSV_OUT = DATA_ROOT/"ss27k_imageai_labels.csv"

In [6]:
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( MODELS_RETINA )
detector.loadModel()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
class StreetstyleImageStream():
    def __init__( self, csv, image_dir ):
        self.df = pd.read_csv( csv )
        self.image_dir = image_dir
        
    def __len__( self ):
        return len( self.df )
        
    def __iter__( self ):
        done = set()
        for _, item in self.df[["old_image", "x1", "x2", "y1", "y2"]].iterrows():
            if item["old_image"] in done:
                continue
            else:
                done.add( item["old_image"] )  
            
            all_bboxes = self.df[["id", "x1", "x2", "y1", "y2"]].loc[self.df["old_image"] == item["old_image"]]
            
            bboxes = dict() 
            for _, item2 in all_bboxes.iterrows():
                bbox = { "x1": item2["x1"], "y1": item2["y1"], "x2": item2["x2"], "y2": item2["y2"] }
                bboxes[item2["id"]] = bbox
            
            yield self.image_dir/item["old_image"], bboxes

In [8]:
def get_ioa(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    
    ioa = intersection_area / float(bb1_area)
    
    assert ioa >= 0.0
    assert ioa <= 1.0
    return ioa

In [9]:
custom = detector.CustomObjects( person=True )
image_stream = StreetstyleImageStream( CSV, IMAGE_DIR )

In [15]:
def load():
    with open( PICKLES/"matches.pkl", "rb" ) as f:
        matches = pickle.load(f)
    
    with open( PICKLES/"unknowns.pkl", "rb" ) as f:
        unknowns = pickle.load(f)
        
    with open( PICKLES/"done.pkl", "rb" ) as f:
        done = pickle.load(f)
        
    with open( PICKLES/"detections.pkl", "rb" ) as f:
        det = pickle.load(f)
        
    return matches, unknowns, done, det
        
def save( matches, unknowns, done, det ):
    with open( PICKLES/"matches.pkl", "wb" ) as f:
        pickle.dump( matches, f, pickle.HIGHEST_PROTOCOL )
        
    with open( PICKLES/"unknowns.pkl", "wb" ) as f:
        pickle.dump( unknowns, f, pickle.HIGHEST_PROTOCOL )
        
    with open( PICKLES/"done.pkl", "wb" ) as f:
        pickle.dump( done, f, pickle.HIGHEST_PROTOCOL )
        
    with open( PICKLES/"detections.pkl", "wb" ) as f:
        pickle.dump( det, f, pickle.HIGHEST_PROTOCOL )

In [14]:
try:
    matches, unknowns, done, det = load()
except FileNotFoundError:
    matches = dict()
    unknowns = dict()
    done = set()
    det = dict()
try:
    total = len( image_stream )
    for n, ( im, bboxes ) in enumerate( image_stream ):
        if n%(int(total*0.05)) == 0:
            print( "{}% done".format( round( ( n / total ) * 100.0 ) ) )
        
        if im in done:
            continue
            
        _, detections = detector.detectCustomObjectsFromImage( custom_objects=custom, input_image=im, 
                                                               minimum_percentage_probability=50,
                                                               output_type="array" )
        
        det[im] = detections

        index_to_id = [ bid for bid, _ in bboxes.items() ]
        ioa_scores = np.zeros( shape=( len( detections ), len( bboxes ) ), dtype=float )
        for n, detection in enumerate( detections ):
            prob = detection["percentage_probability"]
            detection_bbox = detection["box_points"]
            detection_bbox = { "x1": detection_bbox[0], "y1": detection_bbox[1], "x2": detection_bbox[2], "y2": detection_bbox[3] }

            for n2, ( bid, bbox ) in enumerate( bboxes.items() ):
                ioa_scores[n][n2] = get_ioa( bbox, detection_bbox )

        while np.sum( ioa_scores ) > 0.0:
            for n, ioa in enumerate( ioa_scores ): 
                ioa_max_ind = np.argmax( ioa )
                ioa_max = ioa[ioa_max_ind]
                if ioa_max == 0.0:
                    continue

                ioa_bid_max_ind = np.argmax( ioa_scores[:,ioa_max_ind] )
                if ioa_max == ioa_scores[ioa_bid_max_ind,ioa_max_ind]:
                    #print( "{} matched, IoA: {}".format( index_to_id[ioa_max_ind], ioa_max ) )
                    matches[index_to_id[ioa_max_ind]] = detections[n]["box_points"]
                    ioa_scores[n] = 0.0
                    ioa_scores[:,ioa_max_ind] = 0.0

        for bid in index_to_id:
            if bid not in matches:
                unknowns[bid] = [ bboxes[bid], detections ]
                print( "Could not find a bbox for {}".format( bid ) ) 
                
        done.add( im )
except Exception as e:
    raise e
finally:
    save( matches, unknowns, done, det )

0% done
Could not find a bbox for 116007
Could not find a bbox for 629589
Could not find a bbox for 1391283
Could not find a bbox for 8694374
Could not find a bbox for 1906560
Could not find a bbox for 388034
Could not find a bbox for 8651398
Could not find a bbox for 2758114
5% done
Could not find a bbox for 388251
Could not find a bbox for 1967378
Could not find a bbox for 85400
Could not find a bbox for 3735170
Could not find a bbox for 2331708
Could not find a bbox for 1967307
10% done
Could not find a bbox for 24265
Could not find a bbox for 8957239
Could not find a bbox for 3739654
Could not find a bbox for 3813517
Could not find a bbox for 3809717
Could not find a bbox for 3758360
Could not find a bbox for 3728972
15% done
Could not find a bbox for 8961292
Could not find a bbox for 3727668
Could not find a bbox for 3757895
Could not find a bbox for 3740427
Could not find a bbox for 3728632
Could not find a bbox for 3758120
Could not find a bbox for 3727894
Could not find a bbox 

In [15]:
save( matches, unknowns, done, det )
print( len( matches ), len( unknowns ), len( done ), len( det ) )

25076 131 23176 23176


In [17]:
matches, unknowns, done, det = load()

In [30]:
df = pd.read_csv( CSV )

with open( CSV_OUT, "w" ) as f:
    f.write( "id,old_image,image,x1,y1,x2,y2,labels\n" )
    for bid, bbox in matches.items():
        old_image, image, labels = df[["old_image","image","label"]].loc[df["id"] == bid].to_numpy()[0]
        f.write( "{},{},{},{},{},{},{},{}\n".format( bid, old_image, image, bbox[0], bbox[1], bbox[2], bbox[3], labels ) )